In [1]:
import rasterio as rio
from rasterio.windows import Window
import numpy as np
from numba import njit
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import glob
import os
from numba.core import types
from numba.typed import Dict
from collections import Counter

In [2]:
def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob.glob(''.join(map(either, pattern)))

In [31]:
watershed_path = '/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers/'
watersheds = [os.path.basename(f.path) for f in os.scandir(watershed_path) if f.is_dir()]

In [35]:
@njit
def rasterstats(combvals, elevals, elenodata, slopevals, slopenodata, vegvals, vegnodata):
    
    unique_vals = np.unique(combvals)
    unique_vals = unique_vals[unique_vals!=combnodata]
    
    slope_dict = Dict.empty(key_type=types.int32,value_type=types.float64)
    ele_dict = Dict.empty(key_type=types.int32,value_type=types.float64)
    veg_dict = Dict.empty(key_type=types.int32,value_type=types.float64)

    avgslope_dict = Dict.empty(key_type=types.int32,value_type=types.float64)
    avgele_dict = Dict.empty(key_type=types.int32,value_type=types.float64)
    avgveg_dict = Dict.empty(key_type=types.int32,value_type=types.float64)

    slopecount_dict = Dict.empty(key_type=types.int32,value_type=types.int64)
    elecount_dict = Dict.empty(key_type=types.int32,value_type=types.int64)
    vegcount_dict = Dict.empty(key_type=types.int32,value_type=types.int64)

    for val in unique_vals:
        slope_dict[val] = 0
        ele_dict[val] = 0
        veg_dict[val] = 0

        slopecount_dict[val] = 0
        elecount_dict[val] = 0
        vegcount_dict[val] = 0

    xmax, ymax = combvals.shape
    for x in range(0,xmax):
        for y in range(0,ymax):
            val = combvals[x,y]
            if val != combnodata:
                slopeval = slopevals[x,y]
                vegval = vegvals[x,y]
                eleval = elevals[x,y]

                if slopeval != slopenodata:
                    slope_dict[val] += slopeval
                    slopecount_dict[val] += 1
                if vegval != vegnodata:
                    veg_dict[val] += vegval
                    vegcount_dict[val] += 1
                if eleval != elenodata:
                    ele_dict[val] += eleval
                    elecount_dict[val] += 1

    for val in unique_vals:
        if slopecount_dict[val]> 0 :
            avgslope_dict[val] = slope_dict[val]/slopecount_dict[val]
        if elecount_dict[val] > 0:
            avgele_dict[val] = ele_dict[val]/elecount_dict[val]
        if vegcount_dict[val] > 0:
            avgveg_dict[val] = veg_dict[val]/vegcount_dict[val]
    
    return(avgslope_dict,avgele_dict,avgveg_dict)

In [36]:
def read_window_generator(hru_numrows,hru_numcols,blocksize):
    colrange = range(0,hru_numcols,blocksize)
    rowrange = range(0,hru_numrows,blocksize)
    for col in colrange:
        for row in rowrange:
            topleft = [col, row]
            
            if col == max(colrange):
                br_col = hru_numcols
                block_width = hru_numcols - max(colrange)
            else:
                br_col = col + blocksize
                block_width = blocksize
            if row == max(rowrange):
                br_row = hru_numrows
                block_height = hru_numrows - max(rowrange)
            else:
                br_row = row + blocksize
                block_height = blocksize
                
            bottomright = [br_col, br_row]
            
            col_offset = col
            row_offset = row
            
            read_window =  Window(col_offset, row_offset, block_width, block_height)
            
            yield(read_window)

In [37]:
#stolen directly from stackoverflow, dunno how it works
def dictmean(dictlist):
    sums = Counter()
    counters = Counter()
    for itemset in dictlist:
        sums.update(itemset)
        counters.update(itemset.keys())

    ret = {x: float(sums[x])/counters[x] for x in sums.keys()}
    return(ret)

In [38]:
watersheds = [watersheds[5]]

IndexError: list index out of range

In [39]:
for watershed in watersheds:
    
    elefile = insensitive_glob(f'/media/eric/data/data/auckland/hru_creation/nodata_interpolation/*levation_{watershed}.tif.tif')[0]
    slopefile = insensitive_glob(f'/media/eric/data/data/auckland/hru_creation/nodata_interpolation/*lope_{watershed}.tif.tif')[0]
    vegfile = insensitive_glob(f'/media/eric/data/data/auckland/hru_creation/nodata_interpolation/*egetation_{watershed}.tif.tif')[0]
    combfile = insensitive_glob(f'/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers_Combine/{watershed}/*ombine.tif')[0]
    combdbf = insensitive_glob(f'/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers_Combine/{watershed}/*ombine.tif.vat.dbf')[0]

    if not os.path.exists(elefile):
        print(f'ERROR IN FILESEARCH: {watershed}')
    if not os.path.exists(slopefile):
        print(f'ERROR IN FILESEARCH: {watershed}')
    if not os.path.exists(vegfile):
        print(f'ERROR IN FILESEARCH: {watershed}')
    if not os.path.exists(combfile):
        print(f'ERROR IN FILESEARCH: {watershed}')
        
    with rio.open(combfile) as src:
        hru_numrows, hru_numcols = src.shape
    
    read_windows = read_window_generator(hru_numrows,hru_numcols,blocksize=4000)
    
    slopelist = []
    elelist = []
    veglist = []
    
    for read_window in tqdm(read_windows):

        with rio.open(combfile) as src:
            combvals = src.read(1, window=read_window)
            combnodata = src.nodatavals[0]
        with rio.open(elefile) as src:
            elevals = src.read(1, window=read_window)
            elenodata = src.nodatavals[0]
        with rio.open(slopefile) as src:
            slopevals = src.read(1, window=read_window)
            slopenodata = src.nodatavals[0]
        with rio.open(vegfile) as src:
            vegvals = src.read(1, window=read_window)
            vegnodata = src.nodatavals[0]

        avgslope_dict,avgele_dict,avgveg_dict=rasterstats(combvals, elevals, elenodata, slopevals, slopenodata, vegvals, vegnodata)
        slopelist.append(avgslope_dict)
        elelist.append(avgele_dict)
        veglist.append(avgveg_dict)
        
    avgslope_dict = dictmean(slopelist)
    avgele_dict = dictmean(elelist)
    avgveg_dict = dictmean(veglist)
    
    stats_table = pd.DataFrame([avgslope_dict,avgele_dict,avgveg_dict]).T
    stats_table.columns = ['Avg_Slope','Avg_Elev','Avg_Veg']
    stats_table.to_csv(f'/home/eric/projects/auckland/hru_creation/hru_stats/{watershed}_stats.csv')
    print(f'{watershed} calculations complete.')

108it [04:11,  2.33s/it]


Islands calculations complete.


In [27]:
insensitive_glob(f'/media/eric/data/data/auckland/hru_creation/nodata_interpolation/*levation_{watershed}.tif.tif')

[]

In [21]:
insensitive_glob(f'/media/eric/data/data/auckland/hru_creation/nodata_interpolation/*levation_{watershed}.tif.tif')

['/media/eric/data/data/auckland/hru_creation/nodata_interpolation/elevation_islands.tif.tif']

In [52]:
unique_vals, ele_avg, slope_avg, veg_avg = avg_noncategoricals(combvals, elevals, slopevals, vegvals)
StatsDF = pd.DataFrame({'Comb_ID':unique_vals, 'Elevation':ele_avg,'Slope':slope_avg,'Vegetation':veg_avg})
Combine_Table = gpd.read_file(combdbf)
Combine_Table_With_Stats = Combine_Table.merge(right=StatsDF,how='inner',left_on='Value',right_on='Comb_ID')
Combine_Table_With_Stats = Combine_Table_With_Stats.drop(columns=['geometry','Comb_ID'])
Combine_Table_With_Stats.to_csv(f'/home/eric/projects/auckland/hru_creation/{watershed}_HRU_Table_With_Stats.csv')

100%|██████████| 12344/12344 [1:59:58<00:00,  1.71it/s] 


In [53]:
StatsDF = pd.DataFrame({'Comb_ID':unique_vals, 'Elevation':ele_avg,'Slope':slope_avg,'Vegetation':veg_avg})

In [56]:
Combine_Table = gpd.read_file(combdbf)

In [58]:
Combine_Table_With_Stats = Combine_Table.merge(right=StatsDF,how='inner',left_on='Value',right_on='Comb_ID')

In [61]:
Combine_Table_With_Stats = Combine_Table_With_Stats.drop(columns=['geometry','Comb_ID'])

In [62]:
Combine_Table_With_Stats.to_csv(f'/home/eric/projects/auckland/hru_creation/{watershed}_HRU_Table_With_Stats.csv')

In [23]:
StatsDF.to_csv('/home/eric/projects/auckland/hru_creation/Tamaki_HRU_Table.csv',index=False)

In [6]:
StatsDF = pd.read_csv('/home/eric/projects/auckland/hru_creation/Tamaki_HRU_Table.csv')

In [10]:
Combine_Table_With_Stats = Combine_Table.merge(right=StatsDF,how='inner',left_on='Value',right_on='Comb_ID')

In [11]:
Combine_Table_With_Stats.to_csv('/home/eric/projects/auckland/hru_creation/Tamaki_HRU_Table_With_Stats.csv')

In [2]:
from dbfread import DBF

ModuleNotFoundError: No module named 'dbfread'

In [3]:
import sys
sys.executable

'/home/eric/miniconda3/envs/distgeo/bin/python'